# Example of AI Agent

#### The Notebook demonstrates how to create a simple AI Agent that uses vLLM & OpenAI API to communicate with LLM Model.

### Importing Required Libraries

In [2]:
import os
from openai import OpenAI
import json
from vllm import SamplingParams

In [2]:
import os
from openai import OpenAI
import json
from vllm import SamplingParams

## Deploying the LLM using vLLM

Start deploying the LLM (/deepseek-ai/deepseek-llm-7b-chat) using vLLM in the Jupyter notebook:

### Start the vLLM server 

Open a new tab in this Jypyter server, click on the terminal icon to open a new terminal, then copy the following command to launch the vLLM server:

```bash
HIP_VISIBLE_DEVICES=0 vllm serve /home/user/Models/deepseek-ai/deepseek-llm-7b-chat \
        --gpu-memory-utilization 0.9 \
        --swap-space 16 \
        --disable-log-requests \
        --dtype float16 \
        --max-model-len 2048 \
        --tensor-parallel-size 1 \
        --host 0.0.0.0 \
        --port 3000 \
        --num-scheduler-steps 10 \
        --max-num-seqs 128 \
        --max-num-batched-tokens 2048 \
        --max-model-len 2048 \
        --distributed-executor-backend "mp"
```

#### Configure the BASE_URL & MODEL_PATH to input to AI Agent 

In [3]:
BASE_URL = f"http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"

#### Create OpenAI Client

In [4]:
client = OpenAI(api_key="NULL", base_url=BASE_URL, timeout=None, max_retries=0)

#### Create an AI Agent that takes OpenAI Client & Text Input and follows pre-defined Instructions to output the response

In [5]:
class AI_AGENT:
    def __init__(self, client, MODEL_PATH):
        self.base_url = BASE_URL
        self.model_path = MODEL_PATH

    def parse_email(self, email_text):
        response = client.chat.completions.create(
            model=self.model_path,
            temperature=0.0,
            messages=[{
                "role": "user",
                "content": f"""
                Yor are an Agent that helps in scheduling meetings.
                Your job is to extracts Email ID's and Meeting Duration.
                You should return :
                1. List of email id's of participants (comma-separated).
                2. Meeting duration in minutes.
                3. Time constraints (e.g., 'next week').
                If the List of email id's of participants are just names, then append @amd.com at the end and return. 
                Return as json with 'participants', 'time_constraints' & 'meeting_duration'.
                Stricty follow the instructions. Strictly return dict with participents email id's, time constraints & meeting duration in minutes only. 
                Don not add any other instrctions or information. 
                
                Email: {email_text}
                
                """
            }]
        )
        return json.loads(response.choices[0].message.content)

In [6]:
ai_agent = AI_AGENT(client, MODEL_PATH)

In [8]:
email = """
Hi team, let's meet about AI Project next week for half hour. 
Attendees: Ajith(ajith@amd.com), Prakash(prakash@gmail.com), Sirra(sirra@gmail.com). 
"""
meeting_json = ai_agent.parse_email(email)
meeting_json

{'participants': 'ajith@amd.com,prakash@gmail.com,sirra@gmail.com',
 'time_constraints': 'next week',
 'meeting_duration': 30}

In [7]:
meeting_json

{'participants': 'ajith@amd.com,prakash@gmail.com,sirra@gmail.com',
 'time_constraints': 'next week',
 'meeting_duration': 30}

In [68]:
import json
from datetime import datetime, timedelta

class AI_AGENT:
    def __init__(self, client, MODEL_PATH):
        self.base_url = BASE_URL  # Define BASE_URL in your environment
        self.model_path = MODEL_PATH
        self.client = client

    def parse_email(self, email_text):
        response = self.client.chat.completions.create(
            model=self.model_path,
            temperature=0.0,
            messages=[{
                "role": "user",
                "content": f"""
                You are an AI assistant tasked with analyzing emails to determine the context and severity of any issues mentioned.
                Your job is to:
                1. Determine the severity of any issues (classified as 'Low', 'Medium', 'High', or 'None' if no issue is present).
                2. Provide a reasoning that describes the context of the email and explains the severity classification.
                3.Provide the duration of meet in minutes (based on the severity and context of meet)(eg 30 mins ,45 mins etc..) .
                Return a JSON object with 'severity' , 'reasoning' and 'duration' fields only.
                Strictly follow the instructions. Do not add any other information.
                
                Email: {email_text}
                """
            }]
        )
        return json.loads(response.choices[0].message.content)

def process_email(json_input):
    # Parse JSON input
    data = json.loads(json_input) if isinstance(json_input, str) else json_input
    
    # Extract relevant fields
    email_content = data.get("EmailContent", "")
    subject = data.get("Subject", "")
    start_time_str = data.get("Datetime", "")
    
    # Parse start time
    try:
        start_time = datetime.strptime(start_time_str, "%d-%m-%YT%H:%M:%S")
    except ValueError:
        return {"error": "Invalid datetime format. Expected DD-MM-YYYYTHH:MM:SS"}
    
    # Calculate end time (14 days from start)
    end_time = start_time + timedelta(days=14)
    
    # Initialize AI agent with provided client
    ai_agent = AI_AGENT(client, MODEL_PATH)
    
    # Analyze email for context and severity
    email_text = f"Subject: {subject}\nContent: {email_content}"
    analysis = ai_agent.parse_email(email_text)
    
    # Format response
    return {
        "severity": analysis["severity"],
        "reasoning": analysis["reasoning"],
        "duration":analysis["duration"],
        "start_time": start_time.strftime("%d-%m-%YT%H:%M:%S"),
        "end_time": end_time.strftime("%d-%m-%YT%H:%M:%S")
    }

In [69]:
process_email({ "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
})

INFO:httpx:HTTP Request: POST http://localhost:3000/v1/chat/completions "HTTP/1.1 200 OK"


{'severity': 'Medium',
 'reasoning': 'The email is about a project status update for the Agentic AI project, which indicates that there might be some updates or issues that need to be discussed. The mention of a meeting also suggests that it is important to address these issues promptly.',
 'duration': '30 mins',
 'start_time': '09-07-2025T12:34:55',
 'end_time': '23-07-2025T12:34:55'}

In [8]:
email = """
Hi Team, let's meet about AI Project next tuesday for an hour. 
Attendees: Ajith, Prakash & Sirra. 
"""
meeting_json = ai_agent.parse_email(email.lower())
meeting_json

{'participants': 'ajith@amd.com,prakash@amd.com,sirra@amd.com',
 'time_constraints': 'next tuesday',
 'meeting_duration': 60}

In [70]:
input={
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

In [47]:
ai_agent = AI_AGENT(client, MODEL_PATH)


In [48]:
meeting_json = ai_agent.process_meeting_request({
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
})

FileNotFoundError: [Errno 2] No such file or directory: './Keys/team_member1.token'

In [24]:
meeting_json

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033a5',
 'Datetime': '09-07-2025T12:34:55',
 'Location': 'IIT Mumbai',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'usertwo.amd@gmail.com'},
  {'email': 'userthree.amd@gmail.com'}],
 'Subject': 'Agentic AI Project Status Update',
 'EmailContent': "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.",
 'Duration_mins': '30',
 'severity': 'medium',
 'parsed_participants': ['team_member1@amd.com',
  'team_member2@amd.com',
  'team_member3@amd.com']}

In [72]:
import json
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/" + user.split("@")[0] + ".token"
    try:
        user_creds = Credentials.from_authorized_user_file(token_path)
        calendar_service = build("calendar", "v3", credentials=user_creds)
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=start,
            timeMax=end,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        
        for event in events:
            attendee_list = []
            try:
                for attendee in event.get("attendees", []):
                    attendee_list.append(attendee['email'])
            except:
                attendee_list.append("SELF")
            # Handle both dateTime and date fields
            start_time = event["start"].get("dateTime", event["start"].get("date", ""))
            end_time = event["end"].get("dateTime", event["end"].get("date", ""))
            if not start_time or not end_time:
                continue  # Skip events with invalid time formats
            events_list.append({
                "StartTime": start_time,
                "EndTime": end_time,
                "NumAttendees": len(set(attendee_list)),
                "Attendees": list(set(attendee_list)),
                "Summary": event.get("summary", "")
            })
    except FileNotFoundError:
        events_list.append({"error": f"Token file not found for {user}"})
    except Exception as e:
        events_list.append({"error": f"Failed to retrieve events for {user}: {str(e)}"})
    return events_list

def populate_events_in_json(json_input):
    # Parse JSON input
    data = json.loads(json_input) if isinstance(json_input, str) else json_input.copy()
    
    # Extract Datetime and compute time range
    start_time_str = data.get("Datetime", "")
    try:
        start_dt = datetime.strptime(start_time_str, "%d-%m-%YT%H:%M:%S")
        start_time_rfc3339 = start_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
        end_dt = start_dt + timedelta(days=14)
        end_time_rfc3339 = end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    except ValueError:
        return {"error": "Invalid datetime format. Expected DD-MM-YYYYTHH:MM:SS"}
    
    # Extract Attendees
    attendees = data.get("Attendees", [])
    
    # Update events for each attendee
    for attendee in attendees:
        email = attendee.get("email", "")
        if not email:
            attendee["events"] = []
            continue
        # Fetch events for the user
        events = retrive_calendar_events(email, start_time_rfc3339, end_time_rfc3339)
        # Filter out error entries
        valid_events = [event for event in events if "error" not in event]
        attendee["events"] = valid_events
    
    return data

In [83]:
import json
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil import tz
class AI_AGENT:
    def __init__(self, client, MODEL_PATH):
        self.base_url = "[invalid url]"  # Placeholder
        self.model_path = MODEL_PATH
        self.client = client

    def give_time(self, json_input):
        # Parse JSON input
        data = json.loads(json_input) if isinstance(json_input, str) else json_input
        subject = data.get("Subject", "")
        email_content = data.get("EmailContent", "")
        datetime_str = data.get("Datetime", "")
        from_email = data.get("From", "")
        attendees = data.get("Attendees", [])

        # Add From to attendees if not present
        if from_email and not any(a["email"] == from_email for a in attendees):
            attendees.append({"email": from_email, "events": []})

        # Convert datetime_str to datetime object with IST timezone
        try:
            start_dt = datetime.strptime(datetime_str, "%d-%m-%YT%H:%M:%S").replace(tzinfo=tz.gettz("Asia/Kolkata"))
            end_dt = start_dt + timedelta(days=14)
        except ValueError:
            return {"error": "Invalid datetime format. Expected DD-MM-YYYYTHH:MM:SS"}

        # Use LLM to get severity and duration
        prompt = f"""
        Analyze the following email content to determine the severity of the issue (low, medium, high) and suggest a suitable duration in minutes for a meeting to discuss it. The duration should be between 15 and 120 minutes. Return your answer in JSON format: {{"severity": "low|medium|high", "duration": <minutes>}}.

        Subject: {subject}
        Content: {email_content}
        """
        try:
            response = self.client.chat.completions.create(
                model=self.model_path,
                temperature=0.0,
                messages=[{"role": "user", "content": prompt}]
            )
            llm_output = response.choices[0].message.content
            llm_result = json.loads(llm_output)
            severity = llm_result["severity"]
            duration_minutes = int(llm_result["duration"])
            if severity not in ["low", "medium", "high"] or not (15 <= duration_minutes <= 120):
                raise ValueError("Invalid LLM response")
        except Exception as e:
            # Fallback
            severity = "low"  # Based on "status update" and "30 minutes"
            duration_minutes = 30
            print(f"LLM error: {str(e)}")

        # Find a free slot
        def is_working_hours(dt):
            ist = tz.gettz("Asia/Kolkata")
            dt_ist = dt.astimezone(ist)
            if dt_ist.weekday() >= 5:  # Saturday or Sunday
                return False
            hour = dt_ist.hour
            minute = dt_ist.minute
            return 9 <= hour < 18 or (hour == 18 and minute == 0)

        def has_conflict(start, end, events):
            for event in events:
                if event.get("Summary") in ["Off Hours", "Weekend"]:
                    continue
                event_start = parse(event["StartTime"])
                event_end = parse(event["EndTime"])
                if not (end <= event_start or start >= event_end):
                    return True
            return False

        current_dt = start_dt
        duration = timedelta(minutes=duration_minutes)
        found_slot = False
        max_search_dt = end_dt + timedelta(days=30)  # Search up to 30 days after end_dt

        while current_dt < max_search_dt:
            slot_end = current_dt + duration
            if is_working_hours(current_dt):
                conflict = False
                for attendee in attendees:
                    events = attendee.get("events", [])
                    if has_conflict(current_dt, slot_end, events):
                        conflict = True
                        break
                if not conflict:
                    found_slot = True
                    start_time = current_dt.isoformat()
                    end_time = slot_end.isoformat()
                    reason = f"Scheduled meeting at {start_time} for {duration_minutes} minutes based on {severity} severity. This slot is within working hours (9:00 AM to 6:00 PM IST, Monday to Friday) and has no conflicts for all participants."
                    if current_dt > end_dt:
                        reason += " Note: This slot is outside the initial 14-day range but is the earliest available."
                    break
            current_dt += timedelta(minutes=30)

        if not found_slot:
            return {
                "severity": severity,
                "duration": str(duration_minutes),
                "start_time": "",
                "end_time": "",
                "reason": "No available slot found within 30 days after the end date."
            }

        return {
            "severity": severity,
            "duration": str(duration_minutes),
            "start_time": start_time,
            "end_time": end_time,
            "reason": reason
        }

In [84]:
ai_agent.give_time({'Request_id': '6118b54f-907b-4451-8d48-dd13d76033a5',
 'Datetime': '09-07-2025T12:34:55',
 'Location': 'IIT Mumbai',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'usertwo.amd@gmail.com',
   'events': [{'StartTime': '2025-07-09T18:00:00+05:30',
     'EndTime': '2025-07-10T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-10T18:00:00+05:30',
     'EndTime': '2025-07-11T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-11T18:00:00+05:30',
     'EndTime': '2025-07-12T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-12',
     'EndTime': '2025-07-13',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-12T18:00:00+05:30',
     'EndTime': '2025-07-13T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-13',
     'EndTime': '2025-07-14',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-13T18:00:00+05:30',
     'EndTime': '2025-07-14T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-14T18:00:00+05:30',
     'EndTime': '2025-07-15T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-15T09:00:00+05:30',
     'EndTime': '2025-07-15T16:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'AMD AI Workshop'},
    {'StartTime': '2025-07-15T18:00:00+05:30',
     'EndTime': '2025-07-16T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-16T18:00:00+05:30',
     'EndTime': '2025-07-17T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-17T18:00:00+05:30',
     'EndTime': '2025-07-18T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-18T18:00:00+05:30',
     'EndTime': '2025-07-19T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-19',
     'EndTime': '2025-07-20',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-19T18:00:00+05:30',
     'EndTime': '2025-07-20T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-20',
     'EndTime': '2025-07-21',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-20T18:00:00+05:30',
     'EndTime': '2025-07-21T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-21T18:00:00+05:30',
     'EndTime': '2025-07-22T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-22T18:00:00+05:30',
     'EndTime': '2025-07-23T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-23T18:00:00+05:30',
     'EndTime': '2025-07-24T09:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'}]},
  {'email': 'userthree.amd@gmail.com',
   'events': [{'StartTime': '2025-07-09T16:00:00+05:30',
     'EndTime': '2025-07-10T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-10T16:00:00+05:30',
     'EndTime': '2025-07-11T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-11T16:00:00+05:30',
     'EndTime': '2025-07-12T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-12',
     'EndTime': '2025-07-13',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-12T16:00:00+05:30',
     'EndTime': '2025-07-13T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-13',
     'EndTime': '2025-07-14',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-13T16:00:00+05:30',
     'EndTime': '2025-07-14T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-14T09:00:00+05:30',
     'EndTime': '2025-07-14T10:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': '1V1 Team Member'},
    {'StartTime': '2025-07-14T16:00:00+05:30',
     'EndTime': '2025-07-15T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-15T09:00:00+05:30',
     'EndTime': '2025-07-15T16:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'AMD AI Workshop'},
    {'StartTime': '2025-07-15T16:00:00+05:30',
     'EndTime': '2025-07-16T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-16T10:00:00+05:30',
     'EndTime': '2025-07-16T11:00:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Customer Call - Quarterly update'},
    {'StartTime': '2025-07-16T16:00:00+05:30',
     'EndTime': '2025-07-17T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-17T16:00:00+05:30',
     'EndTime': '2025-07-18T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-18T16:00:00+05:30',
     'EndTime': '2025-07-19T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-19',
     'EndTime': '2025-07-20',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-19T16:00:00+05:30',
     'EndTime': '2025-07-20T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-20',
     'EndTime': '2025-07-21',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-20',
     'EndTime': '2025-07-21',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Weekend'},
    {'StartTime': '2025-07-20T16:00:00+05:30',
     'EndTime': '2025-07-21T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-21T16:00:00+05:30',
     'EndTime': '2025-07-22T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-22T16:00:00+05:30',
     'EndTime': '2025-07-23T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'},
    {'StartTime': '2025-07-23T16:00:00+05:30',
     'EndTime': '2025-07-24T07:30:00+05:30',
     'NumAttendees': 0,
     'Attendees': [],
     'Summary': 'Off Hours'}]}],
 'Subject': 'Agentic AI Project Status Update',
 'EmailContent': "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."})

INFO:httpx:HTTP Request: POST http://localhost:3000/v1/chat/completions "HTTP/1.1 200 OK"


{'severity': 'medium',
 'duration': '30',
 'start_time': '2025-07-09T12:34:55+05:30',
 'end_time': '2025-07-09T13:04:55+05:30',
 'reason': 'Scheduled meeting at 2025-07-09T12:34:55+05:30 for 30 minutes based on medium severity. This slot is within working hours (9:00 AM to 6:00 PM IST, Monday to Friday) and has no conflicts for all participants.'}

In [76]:
ai_agent = AI_AGENT(client, MODEL_PATH)